# Crypto Online Learning - LSTM Training

完整的加密貨幣價格預測模型，使用 LSTM + 在線學習

## 修復事項
✅ 添加目標值縮放
✅ 實現反向縮放
✅ 修複預測輸出

In [ ]:
# [STEP 0] 安裝依賴
!pip install -q yfinance torch pandas numpy matplotlib

In [ ]:
# [STEP 1] 克隆倉庫
import os
if not os.path.exists('crypto-online-learning'):
    !git clone https://github.com/caizongxun/crypto-online-learning.git
os.chdir('crypto-online-learning')
!ls -la

In [ ]:
# [STEP 2] 導入模塊
import torch
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 檢查設備
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')
print(f'PyTorch version: {torch.__version__}')

In [ ]:
# [STEP 3] 導入自定義模塊
from crypto_lstm_model import OnlineLearningPipeline
from data_utils import DataPipeline, MetricsCalculator

print('✅ 模型模塊導入成功')

In [ ]:
# [STEP 4] 下載 BTC 數據
print('下載 BTC 1小時數據...')
btc = yf.download('BTC-USD', period='6mo', interval='1h', progress=False)

# 規範化列名
btc.columns = [col[0].lower() if isinstance(col, tuple) else col.lower() for col in btc.columns]

print(f'✅ 下載了 {len(btc)} 根蠟燭')
print(f'日期範圍: {btc.index[0]} 到 {btc.index[-1]}')
print(f'\nFirst 5 rows:')
print(btc.head())

In [ ]:
# [STEP 5] 特徵工程
print('\n創建技術指標特徵...')
pipeline = DataPipeline(sequence_length=60)

df = btc.copy()
df.columns = [str(col).lower() for col in df.columns]
df = pipeline.create_features(df)
df = df.dropna()

print(f'✅ 創建了 {len(df.columns)} 個特徵')
print(f'數據形狀: {df.shape}')
print(f'Target 範圍: [{df["close"].min():.2f}, {df["close"].max():.2f}]')

In [ ]:
# [STEP 6] 準備序列
print('\n準備 LSTM 序列...')

feature_cols = [col for col in df.columns if col not in ['open', 'high', 'low', 'close', 'volume']]
X = df[feature_cols].values.astype(np.float32)
y = df['close'].values.astype(np.float32)

X_seq, y_seq = pipeline.create_sequences(X, y)

# 分割數據
train_size = int(0.7 * len(X_seq))
val_size = int(0.15 * len(X_seq))
test_size = len(X_seq) - train_size - val_size

X_train = X_seq[:train_size]
y_train = y_seq[:train_size]
X_val = X_seq[train_size:train_size+val_size]
y_val = y_seq[train_size:train_size+val_size]
X_test = X_seq[train_size+val_size:]
y_test = y_seq[train_size+val_size:]

print(f'✅ 序列準備完成')
print(f'  訓練集: {X_train.shape}')
print(f'  驗證集: {X_val.shape}')
print(f'  測試集: {X_test.shape}')

In [ ]:
# [STEP 7] 初始化管道
print('\n初始化在線學習管道...')

input_size = X_train.shape[2]
ol_pipeline = OnlineLearningPipeline(
    input_size=input_size,
    sequence_length=60,
    learning_rate=0.001,
    buffer_capacity=10000,
    device=device
)

# 關鍵：用訓練數據初始化（包括目標值縮放）
X_flat = X_train.reshape(-1, input_size)
ol_pipeline.initialize(X_flat, y_train)

print('✅ 管道初始化完成')

In [ ]:
# [STEP 8] 預訓練
print('\n預訓練模型...')

batch_size = 32
num_epochs = 10  # 增加到 10 個 epoch
training_losses = []

# 填充重放緩衝區
for i in range(len(X_train)):
    sequence = X_train[i]
    target = y_train[i]
    volatility = np.std(sequence)
    priority = 1.0 + volatility
    ol_pipeline.replay_buffer.add(sequence, float(target), priority=priority)

print(f'✅ 重放緩衝區大小: {len(ol_pipeline.replay_buffer)}')

# 訓練
for epoch in range(num_epochs):
    epoch_losses = []
    for batch_idx in range(0, max(1, len(ol_pipeline.replay_buffer) // batch_size)):
        batch = ol_pipeline.replay_buffer.sample(batch_size, use_priority=True)
        loss = ol_pipeline.trainer.train_step(batch)
        if loss is not None:
            epoch_losses.append(loss)
    
    if epoch_losses:
        avg_loss = np.mean(epoch_losses)
        training_losses.append(avg_loss)
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.6f}')

print('✅ 預訓練完成！')

In [ ]:
# [STEP 9] 驗證
print('\n驗證模型...')

ol_pipeline.trainer.model.eval()
val_predictions = []

with torch.no_grad():
    for i in range(len(X_val)):
        sequence = X_val[i]
        # 關鍵：predict_next 會自動反向縮放
        pred = ol_pipeline.predict_next(sequence)
        val_predictions.append(pred)

val_predictions = np.array(val_predictions)

# 計算指標
metrics = MetricsCalculator.calculate_all_metrics(y_val, val_predictions)

print(f'✅ 驗證完成')
print(f'  實際範圍: [{y_val.min():.2f}, {y_val.max():.2f}]')
print(f'  預測範圍: [{val_predictions.min():.2f}, {val_predictions.max():.2f}]')
for metric_name, value in metrics.items():
    if metric_name == 'mape':
        print(f'  {metric_name.upper()}: {value:.2f}%')
    else:
        print(f'  {metric_name.upper()}: {value:.4f}')

In [ ]:
# [STEP 10] 在線學習模擬
print('\n在線學習模擬...')

online_predictions = []
online_y_true = []
online_losses = []

ol_pipeline.trainer.model.train()

for i in range(len(X_test)):
    sequence = X_test[i]
    target = y_test[i]
    
    # 預測（自動反向縮放）
    pred = ol_pipeline.predict_next(sequence)
    online_predictions.append(pred)
    online_y_true.append(float(target))
    
    # 添加到緩衝區並訓練
    volatility = np.std(sequence)
    priority = 1.0 + volatility
    ol_pipeline.replay_buffer.add(sequence, float(target), priority=priority)
    
    if len(ol_pipeline.replay_buffer) >= 16:
        batch = ol_pipeline.replay_buffer.sample(16, use_priority=True)
        loss = ol_pipeline.trainer.train_step(batch)
        if loss is not None:
            online_losses.append(loss)
    
    if (i + 1) % max(1, len(X_test) // 10) == 0:
        avg_loss = np.mean(online_losses[-50:]) if len(online_losses) > 0 else 0
        print(f'Step {i+1}/{len(X_test)} - Loss: {avg_loss:.6f}')

online_predictions = np.array(online_predictions)
online_y_true = np.array(online_y_true)

print('✅ 在線學習完成！')

In [ ]:
# [STEP 11] 評估
print('\n評估在線學習結果...')

online_metrics = MetricsCalculator.calculate_all_metrics(online_y_true, online_predictions)

print(f'✅ 在線學習指標')
print(f'  實際範圍: [{online_y_true.min():.2f}, {online_y_true.max():.2f}]')
print(f'  預測範圍: [{online_predictions.min():.2f}, {online_predictions.max():.2f}]')
for metric_name, value in online_metrics.items():
    if metric_name == 'mape':
        print(f'  {metric_name.upper()}: {value:.2f}%')
    else:
        print(f'  {metric_name.upper()}: {value:.4f}')

In [ ]:
# [STEP 12] 保存模型
import os
os.makedirs('checkpoints', exist_ok=True)
ol_pipeline.save('checkpoints/model.pt')
print('✅ 模型已保存到 checkpoints/model.pt')

In [ ]:
# [STEP 13] 可視化
print('\n生成可視化...')

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 訓練損失
if training_losses:
    axes[0].plot(training_losses, linewidth=2, color='blue')
    axes[0].set_title('Training Loss per Epoch', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss (MSE)')
    axes[0].grid(True, alpha=0.3)

# 驗證集
axes[1].plot(y_val, label='Actual', alpha=0.7, linewidth=1)
axes[1].plot(val_predictions, label='Predicted', alpha=0.7, linewidth=1, linestyle='--')
axes[1].set_title('Validation: Actual vs Predicted', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Sample')
axes[1].set_ylabel('BTC Price')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 在線學習
axes[2].plot(online_y_true, label='Actual', alpha=0.7, linewidth=1)
axes[2].plot(online_predictions, label='Predicted (Online)', alpha=0.7, linewidth=1, linestyle='--')
axes[2].set_title('Online Learning: Actual vs Predicted', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Sample')
axes[2].set_ylabel('BTC Price')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_results_fixed.png', dpi=100, bbox_inches='tight')
plt.show()

print('✅ 結果已保存到 training_results_fixed.png')

In [ ]:
print('\n' + '='*80)
print('✅ 訓練完成！')
print('='*80)
print('\n查看結果：')
print('1. training_results_fixed.png - 訓練可視化')
print('2. checkpoints/model.pt - 保存的模型')